In [ ]:
!pip install gym-super-mario-bros==7.3.0 nes_py

In [1]:
!pip install stable-baselines3[extra]

In [2]:
import gym
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation
import matplotlib.pyplot as plt
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from nes_py.wrappers import JoypadSpace
JoypadSpace.reset = lambda self, **kwargs: self.env.reset(**kwargs) # StackOverflow code to fix a problem when calling state.reset()

In [3]:
SIMPLE_MOVEMENT

[['NOOP'],
 ['right'],
 ['right', 'A'],
 ['right', 'B'],
 ['right', 'A', 'B'],
 ['A'],
 ['left']]

# Random Agent

In [3]:
env = gym.make("SuperMarioBros-v0",apply_api_compatibility=True,render_mode="human")
# Wrap the environment to reduce the action space : 7 instead of 256
env = JoypadSpace(env,SIMPLE_MOVEMENT)

done = True
for step in range(1000):
    if done:
        env.reset()
    action = env.action_space.sample()
    state, reward, done,_, info = env.step(action)
    env.render()
env.close()

C:\Users\guill\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
C:\Users\guill\anaconda3\lib\site-packages\gym\envs\registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(
C:\Users\guill\anaconda3\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Impossible de modifier le mode thread une fois qu’il a été fixé
  warnings.warn(str(err))
C:\Users\guill\anaconda3\lib\site-packages\gym\utils\passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(
C:\Users\guill\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow

# Preprocessing the environment

In [4]:
env = gym.make("SuperMarioBros-v0",apply_api_compatibility=True)
# Wrap the environment to reduce the action space : 7 instead of 256
env = JoypadSpace(env,SIMPLE_MOVEMENT)
# Graycscale the observation space
env= GrayScaleObservation(env, keep_dim=True)
# plt.imshow(env.reset()[0], cmap="Greys")  
# Wrap into the Dummy Environment
env = DummyVecEnv([lambda: env])
# Stack the frames (so the agent can predict the movements of ennemies)
env = VecFrameStack(env,4)
print("OBSERVATION SPACE", str(env.observation_space))
print("ACTION SPACE :",str(env.action_space))
print("RENDER :",str(env.render_mode))

C:\Users\guill\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(


OBSERVATION SPACE Box(0, 255, (240, 256, 4), uint8)
ACTION SPACE : Discrete(7)
RENDER : None


C:\Users\guill\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


# RL Model

In [5]:
import os
from stable_baselines3 import PPO 
from stable_baselines3.common.callbacks import BaseCallback # Saving models

In [6]:
# Callback to save the model every check_freq steps 
# Don't save too often because a trained model is still quite big
class TrainAndLoggingCallback(BaseCallback): 
    def __init__(self,check_freq,save_path,verbose=1):
        super(TrainAndLoggingCallback,self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
        
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model{}'.format(self.n_calls))
            self.model.save(model_path)
        return True

In [7]:
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [8]:
callback = TrainAndLoggingCallback(check_freq=100000,save_path=CHECKPOINT_DIR)

In [9]:
model = PPO("CnnPolicy",env,learning_rate=0.000001,n_steps=512, tensorboard_log=LOG_DIR,verbose=1)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [10]:
model.learn(total_timesteps=1000000,callback=callback)

Logging to ./logs/PPO_1


C:\Users\guill\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 75  |
|    iterations      | 1   |
|    time_elapsed    | 6   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 74           |
|    iterations           | 2            |
|    time_elapsed         | 13           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 8.570496e-06 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.95        |
|    explained_variance   | 0.000873     |
|    learning_rate        | 1e-06        |
|    loss                 | 170          |
|    n_updates            | 10           |
|    policy_gradient_loss | 2.95e-05     |
|    value_loss           | 383          |
------------------------------------------
-----------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 86            |
|    iterations           | 13            |
|    time_elapsed         | 77            |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 2.7403352e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.94         |
|    explained_variance   | -0.00422      |
|    learning_rate        | 1e-06         |
|    loss                 | 0.105         |
|    n_updates            | 120           |
|    policy_gradient_loss | -0.000374     |
|    value_loss           | 0.226         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 86            |
|    iterations           | 14            |
|    time_elapsed         | 82  

------------------------------------------
| time/                   |              |
|    fps                  | 87           |
|    iterations           | 24           |
|    time_elapsed         | 140          |
|    total_timesteps      | 12288        |
| train/                  |              |
|    approx_kl            | 3.379234e-05 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.94        |
|    explained_variance   | 0.00193      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.174        |
|    n_updates            | 230          |
|    policy_gradient_loss | -0.000356    |
|    value_loss           | 0.358        |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 87            |
|    iterations           | 25            |
|    time_elapsed         | 145           |
|    t

...

-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 607           |
|    time_elapsed         | 3267          |
|    total_timesteps      | 310784        |
| train/                  |               |
|    approx_kl            | 0.00010757567 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.2          |
|    explained_variance   | -0.12         |
|    learning_rate        | 1e-06         |
|    loss                 | 0.125         |
|    n_updates            | 6060          |
|    policy_gradient_loss | -0.000336     |
|    value_loss           | 1.02          |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 95            |
|    iterations           | 608           |
|    time_elapsed         | 3273

-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 618           |
|    time_elapsed         | 3331          |
|    total_timesteps      | 316416        |
| train/                  |               |
|    approx_kl            | 0.00043636176 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.33         |
|    explained_variance   | -0.197        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0642        |
|    n_updates            | 6170          |
|    policy_gradient_loss | -0.000277     |
|    value_loss           | 0.141         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 94            |
|    iterations           | 619           |
|    time_elapsed         | 3337

------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 629          |
|    time_elapsed         | 3394         |
|    total_timesteps      | 322048       |
| train/                  |              |
|    approx_kl            | 0.0006599921 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.34        |
|    explained_variance   | -0.0548      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.0602       |
|    n_updates            | 6280         |
|    policy_gradient_loss | -0.00155     |
|    value_loss           | 0.119        |
------------------------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 630          |
|    time_elapsed         | 3400         |
|    total_

------------------------------------------
| time/                   |              |
|    fps                  | 94           |
|    iterations           | 640          |
|    time_elapsed         | 3455         |
|    total_timesteps      | 327680       |
| train/                  |              |
|    approx_kl            | 0.0013715683 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.33        |
|    explained_variance   | -0.0675      |
|    learning_rate        | 1e-06        |
|    loss                 | 0.151        |
|    n_updates            | 6390         |
|    policy_gradient_loss | -0.00164     |
|    value_loss           | 0.647        |
------------------------------------------



KeyboardInterrupt



# Testing trained model

In [11]:
model = PPO.load("./train/best_model300000.zip")

In [13]:
env = gym.make("SuperMarioBros-v0",apply_api_compatibility=True,render_mode="human")
env = JoypadSpace(env,SIMPLE_MOVEMENT)
env= GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4)

frames= []
state = env.reset()
for steps in range(1000):
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    frame = env.render()
    frames.append(frame)
env.close()

# To improve the model :

- Reduce the learning_rate
- Train for longer : 1 million epochs should be a minimum


# Optional : Create a mp4 video of the agent

In [14]:
import cv2
env = gym.make("SuperMarioBros-v0",apply_api_compatibility=True,render_mode="rgb_array")
env = JoypadSpace(env,SIMPLE_MOVEMENT)
env= GrayScaleObservation(env, keep_dim=True)
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env,4)

frames= []
state = env.reset()
for steps in range(1000):
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    frame = env.render()
    frames.append(frame)
env.close()

output_video_file = 'agent_demo.mp4'
frame_size = (frames[0].shape[1], frames[0].shape[0])  
fps = 60  
codec = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(output_video_file, codec, fps, frame_size)
for frame in frames:
    bgr_frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    video_writer.write(bgr_frame)

video_writer.release()